In [99]:
API = "api-fxpractice.oanda.com"
STREAM_API = "stream-fxpractice.oanda.com"
ACCESS_TOKEN = "06a31b21abc8fb4cbe9bbd8c5b7b1f4e-260aefea008b878b6bffbfebafc84b9f"
ACCOUNT_ID = "101-004-26291813-001"

In [14]:
import requests
import json
import pandas as pd
import numpy as np

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"

query = {"instruments": "XAU_GBP"}
headers = {"Authorization": "Bearer " + ACCESS_TOKEN}
response = requests.get("https://" + API + PRICING_PATH, headers=headers, params=query)
response.json()

{'time': '2023-07-10T10:41:37.914724736Z',
 'prices': [{'type': 'PRICE',
   'time': '2023-07-10T10:41:36.975613333Z',
   'bids': [{'price': '1503.267', 'liquidity': 2000}],
   'asks': [{'price': '1503.545', 'liquidity': 2000}],
   'closeoutBid': '1503.267',
   'closeoutAsk': '1503.545',
   'status': 'tradeable',
   'tradeable': True,
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'XAU_GBP'}]}

In [25]:
# Creating a function to get the pricing information

def get_pricing(instrument):
    query = {"instruments": instrument}
    response = requests.get("https://" + API + PRICING_PATH, headers=headers, params=query)
    json_response = response.json()

    prices = json_response["prices"][0]["asks"][0]["price"]
    prices = float(prices)

    time_string = json_response["time"]
    time = pd.to_datetime(time_string)
    return time, prices

get_pricing("XAU_GBP")

(Timestamp('2023-07-10 11:02:30.552317152+0000', tz='UTC'), 1504.119)

In [155]:
# Get the historical data

import time 

# Time format is year, month, day
from_time = time.mktime(pd.to_datetime("2023-07-1-00-00-00").timetuple())
to_time = time.mktime(pd.to_datetime("2023-07-10-00-00-00").timetuple())
print(f"window start: {from_time}, window end: {to_time}")

window start: 1688169600.0, window end: 1688947200.0


In [164]:
header = {"Authorization": "Bearer " + ACCESS_TOKEN}
# Granularity is limited for the amount of data available, no more than 3000 entries give or take
query = {"from": from_time, "to": to_time, "granularity": "M5"}

INSTRUMENT = "XAU_GBP"
CANDLES_PATH = f"/v3/accounts/{ACCOUNT_ID}/instruments/{INSTRUMENT}/candles"

response = requests.get("https://" + API + CANDLES_PATH, headers=header, params=query)

def json_to_pd(json):
    json_file = json.json()
    price_json = json_file["candles"]
    times = []
    close_price, high_price, low_price, open_price = [], [], [], []

    for candle in price_json:
        times.append(candle["time"])
        close_price.append(float(candle["mid"]["c"]))
        high_price.append(float(candle["mid"]["h"]))
        low_price.append(float(candle["mid"]["l"]))
        open_price.append(float(candle["mid"]["o"]))
    
    dataframe = pd.DataFrame({"close": close_price, "high": high_price, "low": low_price, "open": open_price})
    dataframe.index = pd.to_datetime(times)
    return dataframe

get_data = json_to_pd(response)
get_data


,close,high,low,open
2023-07-02 22:10:00+00:00,1510.958,1511.306,1510.656,1510.782
2023-07-02 22:15:00+00:00,1510.974,1511.061,1510.681,1510.958
2023-07-02 22:20:00+00:00,1510.740,1511.132,1510.684,1511.010
2023-07-02 22:25:00+00:00,1510.796,1510.942,1510.728,1510.739
2023-07-02 22:30:00+00:00,1510.833,1511.048,1510.811,1510.835
...,...,...,...,...
2023-07-09 23:35:00+00:00,1499.738,1499.738,1499.346,1499.392
2023-07-09 23:40:00+00:00,1499.704,1499.934,1499.612,1499.708
2023-07-09 23:45:00+00:00,1499.644,1500.048,1499.554,1499.664
2023-07-09 23:50:00+00:00,1499.446,1499.674,1499.376,1499.566


In [165]:
# Building the signal generator

def signal_generator(df):
    signal = []
    for index, row in df.iterrows():
        if index == 0:
            signal.append(0)  # No clear pattern for the first row
            continue

        open_price = row["open"]
        close_price = row["close"]
        prev_index = df.index[df.index.get_loc(index) - 1]
        prev_row = df.loc[prev_index]
        prev_open = prev_row["open"]
        prev_close = prev_row["close"]

        # Bearish
        if (
            open_price > close_price
            and prev_open < prev_close
            and close_price < prev_open
            and open_price >= prev_close
        ):
            signal.append(1)
    
        # Bullish
        elif (
            open_price < close_price
            and prev_open > prev_close
            and close_price > prev_open
            and open_price <= prev_close
        ):
            signal.append(2)
    
        # No clear pattern
        else:
            signal.append(0)

    return signal

signal = signal_generator(get_data)
print(signal)

[0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 

In [100]:
# Stream the prcing infomation

import v20

stream_api = v20.Context(hostname=STREAM_API, port=443, token=ACCESS_TOKEN)
stream_api


response = stream_api.pricing.stream(accountID=ACCOUNT_ID, instruments="XAU_GBP", snapshot=True)

for price_data in response.lines:
    print(price_data)



b'{"type":"PRICE","time":"2023-07-10T12:11:06.045638441Z","bids":[{"price":"1505.512","liquidity":2000}],"asks":[{"price":"1505.791","liquidity":2000}],"closeoutBid":"1505.512","closeoutAsk":"1505.791","status":"tradeable","tradeable":true,"instrument":"XAU_GBP"}'
b'{"type":"PRICE","time":"2023-07-10T12:11:14.238257038Z","bids":[{"price":"1505.457","liquidity":2000}],"asks":[{"price":"1505.736","liquidity":2000}],"closeoutBid":"1505.457","closeoutAsk":"1505.736","status":"tradeable","tradeable":true,"instrument":"XAU_GBP"}'
b'{"type":"PRICE","time":"2023-07-10T12:11:16.121741910Z","bids":[{"price":"1505.387","liquidity":2000}],"asks":[{"price":"1505.666","liquidity":2000}],"closeoutBid":"1505.387","closeoutAsk":"1505.666","status":"tradeable","tradeable":true,"instrument":"XAU_GBP"}'
b'{"type":"HEARTBEAT","time":"2023-07-10T12:11:16.860850536Z"}'
b'{"type":"PRICE","time":"2023-07-10T12:11:16.914723628Z","bids":[{"price":"1505.356","liquidity":2000}],"asks":[{"price":"1505.635","liquidi

KeyboardInterrupt: 

In [166]:
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails


# Executing Orders

client = API(access_token=ACCESS_TOKEN)

SLTPRatio = 2
previous_candleR = abs(dfstream['Open'].iloc[-2]-dfstream['Close'].iloc[-2])

SLBuy = float(str(candle.bid.o - previous_candleR))
SLSell = float(str(candle.bid.o + previous_candleR))

TPBuy = float(str(candle.bid.o + (previous_candleR * SLTPRatio)))
TPSell = float(str(candle.bid.o - (previous_candleR * SLTPRatio)))

print(dfstream.iloc[:-1,:])
print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)

#Sell
if signal ==1:
    mo = MarketOrderRequest(instrument="XAU_GBP", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
    r = orders.OrderCreate(accountID=ACCOUNT_ID, data=mo.data)
    rv = client.request(r)
    print(rv)

#Buy
if signal ==2:
    mo = MarketOrderRequest(instrument="XAU_GBP", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
    r = orders.OrderCreate(accountID=ACCOUNT_ID, data=mo.data)
    rv = client.request(r)
    print(rv)

NameError: name 'dfstream' is not defined

In [167]:
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour=8, minute=30, timezone='Europe/London')
scheduler.start()

NameError: name 'trading_job' is not defined